In [205]:
class Car:
    # car_num 車子編號(int)
    # isvertical 垂直/水平(boolean)
    # size 大小(int)
    # istarget 是否是需要移出的那台車(boolean)
    # x,y 以車子的左邊，上面車子的頭(int)
    def __init__(self,car_num,isvertical,size,x,y) :
        self.car_num = car_num
        self.isvertical = isvertical
        self.size = size
        # istarget
        # self.istarget = istarget
        self.x = x
        self.y = y

    # 下面這個method執行car的移動方向
    def car_move(self,direction):
        if direction == "L":
            self.go_left()
        elif direction == "R":
            self.go_right()
        elif direction == "U":
            self.go_up()
        elif direction == "D":
            self.go_down()
    # 水平：
    # 向左移動1格
    def go_left(self):
        self.y = self.y - 1
    # 向右移動1格
    def go_right(self):
        self.y = self.y + 1
    # 垂直
    # 向上移動1格
    def go_up(self):
        self.x = self.x - 1
    # 向下移動1格
    def go_down(self):
        self.x = self.x + 1


In [206]:
class Board:    
    def __init__(self, cars):
        self.size = 5;
        self.exit = (self.size // 2, self.size - 1);
        self.emptySymbol = "x";
        self.cars = cars;
        self.board = [];
        
        if (self.exit[0] != self.cars[0].x):
            print("The puzzle is unsolvable!");
            return;
        
        self.initBoard();
        self.initCars();
        
    def initBoard(self):
        for i in range(self.size):
            self.board.append([]);
            
            for j in range(self.size):
                self.board[i].append(self.emptySymbol);
                
    def initCars(self):
        for car in self.cars:
            self.addCar(car);
    
    # add the car to the board
    def addCar(self, car):
        if (not self.carCrash(car)):
            if (car.isvertical):
                for i in range(car.size):
                    self.board[car.x + i][car.y] = car.car_num;
            else:
                for i in range(car.size):
                    self.board[car.x][car.y + i] = car.car_num;
    
    # check if the spots required by the car is available on the board
    def carCrash(self, car):
        if (car.isvertical):
            for i in range(car.size):
                if (self.occupied(car.x + i, car.y)):
                    return True;
            
            return False;
        
        else:
            for i in range(car.size):
                if (self.occupied(car.x, car.y + i)):
                    return True;
            
            return False;
    
    def solved(self):
        target = self.cars[0];
        
        for i in range(target.y + target.size, self.size):
            if (self.occupied(target.x, i)):
                return False;

        return True;
            
    def printBoard(self):
        for i in range(self.size + 2):
            print("-", end="");
            
        print("");
        
        for i in range(self.size):
            print("|", end="");
            
            for j in range(self.size):
                print(self.board[i][j], end="");
            
            if (i == self.exit[0]):
                print(" ");
            else:
                print("|")
            
        for i in range(self.size + 2):
            print("-", end="");
            
        print("\n");
    
    # check if a spot on the board is occupied by a car
    def occupied(self, x, y):
        return self.board[x][y] != "x";
    
    # check if a move is legal
    def validMove(self, car_num, direction):
        car = self.cars[car_num];
        
        if (direction == "U"):
            if (not car.isvertical):
                #print("invalid direction!");
                return False;
            elif (car.x == 0):
                #print("out of bound!");
                return False;
            elif (self.occupied(car.x - 1, car.y)):
                #print("car crash!");
                return False;
            else:
                return True;
            
        elif (direction == "D"):
            if (not car.isvertical):
                #print("invalid direction!");
                return False;
            elif (car.x + car.size >= self.size):
                #print("out of bound!");
                return False;
            elif (self.occupied(car.x + car.size, car.y)):
                #print("car crash!");
                return False;
            else:
                return True;
            
        elif (direction == "L"):
            if (car.isvertical):
                #print("invalid direction!");
                return False;
            elif (car.y == 0):
                #print("out of bound!");
                return False;
            elif (self.occupied(car.x, car.y - 1)):
                #print("car crash!");
                return False;
            else:
                return True;
            
        elif (direction == "R"):
            if (car.isvertical):
                #print("invalid direction!");
                return False;
            elif (car.y + car.size >= self.size):
                #print("out of bound!");
                return False;
            elif (self.occupied(car.x, car.y + car.size)):
                #print("car crash!");
                return False;
            else:
                return True;
    
    def play(self, car_num, direction):
        if (self.validMove(car_num, direction)):
            car = self.cars[car_num];
            
            if (direction == "U"):
                self.board[car.x - 1][car.y] = car_num;
                self.board[car.x + car.size - 1][car.y] = self.emptySymbol;
                    
            elif (direction == "D"):
                self.board[car.x + car.size][car.y] = car_num;
                self.board[car.x][car.y] = self.emptySymbol;
                    
            elif (direction == "L"):
                self.board[car.x][car.y - 1] = car_num;
                self.board[car.x][car.y + car.size - 1] = self.emptySymbol;
                    
            elif (direction == "R"):
                self.board[car.x][car.y + car.size] = car_num;
                self.board[car.x][car.y] = self.emptySymbol;
                
            car.car_move(direction);

    def stateAfterMove(self, car_num, direction):
        if (self.validMove(car_num, direction)):
            copyCars = [];
            for car in self.cars:
                copyCar = Car(car.car_num, car.isvertical, car.size, car.x, car.y);
                copyCars.append(copyCar);
            
            newBoard = Board(copyCars);
            newBoard.play(car_num, direction);
            
            return newBoard;
        else:
            return None;
        
    def expand(self):
        frontier = [];
        
        for i in range(len(self.cars)):
            for d in ["U", "D", "R", "L"]:
                nextBoard = self.stateAfterMove(i, d);
                
                if (nextBoard is not None):
                    node = (nextBoard, i, d);
                    frontier.append(node);
        
        return frontier;

In [207]:
import heapq

# 算車頭到出口的距離
def manhatten(board):
    return abs(board.cars[0].y - board.exit[1]);

# heuristic function 計算車頭到出口被幾臺車擋住
def heuristic(board):
    heu = 0
    pos = board.cars[0].y
    while pos < board.exit[1]:
        if board.board[board.cars[0].x][pos] != 0 or board.board[board.cars[0].x][pos] != 'x':
            heu = heu + 1
        pos = pos + 1
    return heu


def astar(board):
  ans = []
  visited = []
  start = [] 

  #hx = manhatten(board)
  #hx = heuristic(board)
  hx = manhatten(board) + heuristic(board)
  depth = 0
  fx = hx + depth

  statenode = StateNode(fx, depth, board, start)

  node_list = []
  heapq.heapify(node_list)
  heapq.heappush(node_list, statenode)

  visited.append(board.cars)


  while node_list:
    node = heapq.heappop(node_list)

    if node.state.solved():
      ans = node.path
      return ans

    for B, car_id, dir in node.state.expand():
      
      if B.cars not in visited:

        g = node.depth + 1
        #f = manhatten(B) + g
        #f = heuristic(B) + g
        f = manhatten(B) + heuristic(B) + g
        tmp = StateNode(f, g, B, node.path+[(car_id, dir)]) # 紀錄path

        heapq.heappush(node_list, tmp)
        visited.append(B.cars)
      
  

In [208]:
class StateNode(object):
  def __init__(self, fx, depth, state, path):
    self.fx = fx
    self.depth = depth
    self.state = state
    self.path = path
  
  def __lt__(self, other):
    return self.fx < other.fx

In [209]:
# initial all car 

cars = []
'''novice3
cars.append(Car(0,False,2,2,0)) #red
cars.append(Car(1,False,2,0,1)) #black
cars.append(Car(2,False,2,1,2)) #blue
cars.append(Car(3,True,2,2,2))  #yellow
cars.append(Car(4,False,3,4,2)) #white

'''

cars.append(Car(0,False,2,2,0)) #red
cars.append(Car(1,True, 2,0,1)) #black left
cars.append(Car(2,True, 2,1,2)) #blue left
cars.append(Car(3,True, 2,3,2)) #yellow left
cars.append(Car(4,True, 2,0,3)) #white
cars.append(Car(5,True, 2,2,3)) #black
cars.append(Car(6,True, 2,1,4)) #blue right
cars.append(Car(7,True, 2,3,4)) #yellow right
'''
cars.append(Car(0,False,2,3,0)) #red
cars.append(Car(1,False,2,0,1)) #black up
cars.append(Car(2,False,2,1,0)) #white2 up
cars.append(Car(3,False,2,5,0)) #white2 down
cars.append(Car(4,True, 2,0,3)) #yellow up
cars.append(Car(5,False,3,2,2)) #black down
cars.append(Car(6,True, 3,3,2)) #white3 left
cars.append(Car(7,False,2,0,4)) #blue
cars.append(Car(8,True, 3,1,5)) #white3 right
cars.append(Car(9,True, 2,3,4)) #yellow down
'''


board = Board(cars);
board.printBoard();

ans = astar(board);
print(ans)




-------
|x1x4x|
|x1246|
|00256 
|xx357|
|xx3x7|
-------

[(2, 'U'), (0, 'R'), (5, 'D'), (0, 'R'), (6, 'U')]
